In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

At the first block of every epoch
- Get the list of elected validators (elected_list)
- Get the stakes of validators who were not eligible for election in the last epoch (last block)
- Check if any of these validators present in elected_list

(note: election happen in the last block, however the epos-status changed in the first block)

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getAllElectedValidator():
    method = "hmy_getElectedValidatorAddresses"
    params =[]
    return get_information(method, params)['result']

In [6]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [7]:
def E2_test():
    print("Test-E2: Joining after the election start must not consider the validator ")
    # get the last block in current epoch
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current block number and last block number in current epoch: ", block, last_block)
    while block < last_block:
        block = getBlockNumber()
    print("last block number in the current epoch reaches", block)
    # save the information of status in the last block
    validators = getAllValidatorInformation()

    # get the first block in next epoch
    new_block = block + 1
    while block < last_block:
        block = getBlockNumber()
    print("first block in new epoch reaches: ", block)
    
    # get the elected addresses
    elected = getAllElectedValidator()
    
    ineligible = []
    for i in validators:
        if i['epos-status'] == "currently elected and signing not enough blocks to be eligible for election next epoch"\
        or i['epos-status']== "not eligible to be elected next epoch":
            ineligible.append(i['validator']['address'])
    
    flag = True
    for i in ineligible:
        if i in elected:
            flag = False
            print("Test-E2: Fail")
            print("Validator " + i + "joining after the election was considered for election")

    if flag == True:
        print("Test-E2: Success")


In [8]:
E2_test()

Test-E2: Joining after the election start must not consider the validator 
current block number and last block number in current epoch:  2510 2545
last block number in the current epoch reaches 2545
first block in new epoch reaches:  2545
Test-E2: Success
